**코드 조각 1: 자질 벡터 정의**

In [ ]:
!pip install -U finance-datareader
!pip install ta

In [ ]:
!pip install -U yfinance

In [ ]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
import yfinance as yf
import ta

In [ ]:
COLUMNS_CHART_DATA = ['date', 'open', 'high', 'low', 'close', 'volume']

COLUMNS_TRAINING_DATA = [
    'diffratio', 'open_lastclose_ratio', 'high_close_ratio', 'low_close_ratio', 'close_lastclose_ratio', 'volume_lastvolume_ratio',
    'sma_close_ma5_ratio', 'sma_close_ma10_ratio', 'sma_close_ma20_ratio', 'sma_close_ma60_ratio', 'sma_close_ma120_ratio',
    'sma_volume_ma5_ratio', 'sma_volume_ma10_ratio', 'sma_volume_ma20_ratio', 'sma_volume_ma60_ratio', 'sma_volume_ma120_ratio',
    'ema_close_ma5_ratio', 'ema_close_ma10_ratio', 'ema_close_ma20_ratio', 'ema_close_ma60_ratio', 'ema_close_ma120_ratio',
    'ema_volume_ma5_ratio', 'ema_volume_ma10_ratio', 'ema_volume_ma20_ratio', 'ema_volume_ma60_ratio', 'ema_volume_ma120_ratio',
    'bhb_ratio', 'blb_ratio', 'rsi_ratio', 'sr_ratio', 'macd_ratio', 'obv_ratio', 'adi_ratio',
    'sma_sp_close_ma5_ratio', 'sma_sp_close_ma10_ratio', 'sma_sp_close_ma20_ratio', 'sma_sp_close_ma60_ratio', 'sma_sp_close_ma120_ratio',
    'ema_sp_close_ma5_ratio', 'ema_sp_close_ma10_ratio', 'ema_sp_close_ma20_ratio', 'ema_sp_close_ma60_ratio', 'ema_sp_close_ma120_ratio',
    'sma_dji_close_ma5_ratio', 'sma_dji_close_ma10_ratio', 'sma_dji_close_ma20_ratio', 'sma_dji_close_ma60_ratio', 'sma_dji_close_ma120_ratio',
    'ema_dji_close_ma5_ratio', 'ema_dji_close_ma10_ratio', 'ema_dji_close_ma20_ratio', 'ema_dji_close_ma60_ratio', 'ema_dji_close_ma120_ratio',
    'sma_ixic_close_ma5_ratio', 'sma_ixic_close_ma10_ratio', 'sma_ixic_close_ma20_ratio', 'sma_ixic_close_ma60_ratio', 'sma_ixic_close_ma120_ratio',
    'ema_ixic_close_ma5_ratio', 'ema_ixic_close_ma10_ratio', 'ema_ixic_close_ma20_ratio', 'ema_ixic_close_ma60_ratio', 'ema_ixic_close_ma120_ratio',
    'sma_sp_volume_ma5_ratio', 'sma_sp_volume_ma10_ratio', 'sma_sp_volume_ma20_ratio', 'sma_sp_volume_ma60_ratio', 'sma_sp_volume_ma120_ratio',
    'ema_sp_volume_ma5_ratio', 'ema_sp_volume_ma10_ratio', 'ema_sp_volume_ma20_ratio', 'ema_sp_volume_ma60_ratio', 'ema_sp_volume_ma120_ratio',
    'sma_dji_volume_ma5_ratio', 'sma_dji_volume_ma10_ratio', 'sma_dji_volume_ma20_ratio', 'sma_dji_volume_ma60_ratio', 'sma_dji_volume_ma120_ratio',
    'ema_dji_volume_ma5_ratio', 'ema_dji_volume_ma10_ratio', 'ema_dji_volume_ma20_ratio', 'ema_dji_volume_ma60_ratio', 'ema_dji_volume_ma120_ratio',
    'sma_ixic_volume_ma5_ratio', 'sma_ixic_volume_ma10_ratio', 'sma_ixic_volume_ma20_ratio', 'sma_ixic_volume_ma60_ratio', 'sma_ixic_volume_ma120_ratio',
    'ema_ixic_volume_ma5_ratio', 'ema_ixic_volume_ma10_ratio', 'ema_ixic_volume_ma20_ratio', 'ema_ixic_volume_ma60_ratio', 'ema_ixic_volume_ma120_ratio',
    'sp_close_diff', 'sp_volume_diff', 'dji_close_diff', 'dji_volume_diff', 'ixic_close_diff', 'ixic_volume_diff',
    'buy_strength_ratio', 'sell_strength_ratio'
]

def z_score_normalization(data):
    mean_value = np.mean(data)
    std_dev = np.std(data)
    normalized_data = (data - mean_value) / std_dev
    return normalized_data

def ydatareader(code, start_date, end_date):
    orig_df = yf.download(code, start_date, end_date)
    orig_df.drop(columns=['Adj Close'], inplace=True)
    orig_df.reset_index(drop=False, inplace=True)
    orig_df.columns = ['date', 'open', 'high', 'low', 'close', 'volume']

    prep_df = preprocess(orig_df)

    sp_df = ydatareader_v2('^GSPC', start_date, end_date)
    merge_df = pd.merge_asof(prep_df, sp_df, on='date', direction='nearest')
    dji_df = ydatareader_v2('^DJI', start_date, end_date)
    merge_df = pd.merge_asof(merge_df, dji_df, on='date', direction='nearest')
    ixic_df = ydatareader_v2('^IXIC', start_date, end_date)
    merge_df = pd.merge_asof(merge_df, ixic_df, on='date', direction='nearest')

    date = datetime.strptime(str(start_date), '%Y-%m-%d')
    for i in range(len(merge_df)):
        if merge_df['date'][i] >= datetime(date.year + 1, 1, 1):
            merge_df = merge_df[i:]
            break

    merge_df.reset_index(drop=True, inplace=True)

    return merge_df

def ydatareader_v2(code, start_date, end_date):
    orig_df = yf.download(code, start_date, end_date)
    orig_df.drop(columns=['Adj Close'], inplace=True)
    orig_df.reset_index(drop=False, inplace=True)
    orig_df.columns = ['date', 'open', 'high', 'low', 'close', 'volume']

    prep_df = preprocess_v2(code, orig_df)

    date = datetime.strptime(str(start_date), '%Y-%m-%d')
    for i in range(len(prep_df)):
        if prep_df['date'][i] >= datetime(date.year + 1, 1, 1):
            prep_df = prep_df[i:]
            break

    prep_df.reset_index(drop=True, inplace=True)

    return prep_df

def preprocess(data):
    windows = [5, 10, 20, 60, 120]
    for window in windows:
        data[f'sma_close_ma{window}'] = ta.trend.sma_indicator(close=data['close'], window=window)
        data[f'sma_volume_ma{window}'] = ta.trend.sma_indicator(close=data['volume'], window=window)
        data[f'ema_close_ma{window}'] = ta.trend.ema_indicator(close=data['close'], window=window)
        data[f'ema_volume_ma{window}'] = ta.trend.ema_indicator(close=data['volume'], window=window)

        # Z-점수-정규화
        data[f'sma_close_ma{window}_ratio'] = z_score_normalization(data[f'sma_close_ma{window}'])
        data[f'sma_volume_ma{window}_ratio'] = z_score_normalization(data[f'sma_volume_ma{window}'])
        data[f'ema_close_ma{window}_ratio'] = z_score_normalization(data[f'ema_close_ma{window}'])
        data[f'ema_volume_ma{window}_ratio'] = z_score_normalization(data[f'ema_volume_ma{window}'])

    data['bhb'] = ta.volatility.bollinger_hband(close=data['close'], fillna=True)
    data['blb'] = ta.volatility.bollinger_lband(close=data['close'], fillna=True)
    data['rsi'] = ta.momentum.rsi(close=data['close'], fillna=True)
    data['sr'] = ta.momentum.stoch(close=data['close'], high=data['high'], low=data['low'], fillna=True)
    data['obv'] = ta.volume.on_balance_volume(close=data['close'], volume=data['volume'], fillna=True)
    data['adi'] = ta.volume.acc_dist_index(high=data['high'], low=data['low'], close=data['close'], volume=data['volume'], fillna=True)
    data['macd'] = ta.trend.macd(close=data['close'], fillna=True)

    data['diffratio'] = np.zeros(len(data))
    data.loc[1:, 'diffratio'] = ((data['close'][1:].values - data['close'][:-1].values) / data['close'][:-1].values) * 100
    data['open_lastclose_ratio'] = np.zeros(len(data))
    data.loc[1:, 'open_lastclose_ratio'] = (data['open'][1:].values - data['close'][:-1].values) / data['close'][:-1].values * 100
    data['high_close_ratio'] = (data['high'].values - data['close'].values) / data['close'].values * 100
    data['low_close_ratio'] = (data['low'].values - data['close'].values) / data['close'].values * 100
    data['close_lastclose_ratio'] = np.zeros(len(data))
    data.loc[1:, 'close_lastclose_ratio'] = (data['close'][1:].values - data['close'][:-1].values) / data['close'][:-1].values * 100
    data['volume_lastvolume_ratio'] = np.zeros(len(data))
    data.loc[1:, 'volume_lastvolume_ratio'] = (data['volume'][1:].values - data['volume'][:-1].values) / data['volume'][:-1].replace(to_replace=0, method='ffill').replace(to_replace=0, method='bfill').values
    data['buy_strength'] = np.zeros(len(data))
    data['buy_strength'] = np.log(1 + np.maximum((data['high'] - data['open']) + (data['close'] - data['low']), 0 ) * data['volume']).rolling(5).mean()
    data['sell_strength'] = np.zeros(len(data))
    data['sell_strength'] = np.log(1 + np.maximum((data['high'] - data['close']) + (data['open'] - data['low']), 0) * data['volume']).rolling(5).mean()

    # Z-점수-정규화
    data['bhb_ratio'] = z_score_normalization(data['bhb'])
    data['blb_ratio'] = z_score_normalization(data['blb'])
    data['rsi_ratio'] = z_score_normalization(data['rsi'])
    data['sr_ratio'] = z_score_normalization(data['sr'])
    data['macd_ratio'] = z_score_normalization(data['macd'])
    data['obv_ratio'] = z_score_normalization(data['obv'])
    data['adi_ratio'] = z_score_normalization(data['adi'])
    data['buy_strength_ratio'] = z_score_normalization(data['buy_strength'])
    data['sell_strength_ratio'] = z_score_normalization(data['sell_strength'])

    return data

def preprocess_v2(code, data):
    if code == '^GSPC':
        name = 'sp'
    elif code == '^DJI':
        name = 'dji'
    elif code == '^IXIC':
        name = 'ixic'

    windows = [5, 10, 20, 60, 120]
    for window in windows:
        data[f'sma_{name}_close_ma{window}'] = ta.trend.sma_indicator(close=data['close'], window=window)
        data[f'ema_{name}_close_ma{window}'] = ta.trend.ema_indicator(close=data['close'], window=window)
        data[f'sma_{name}_volume_ma{window}'] = ta.trend.sma_indicator(close=data['volume'], window=window)
        data[f'ema_{name}_volume_ma{window}'] = ta.trend.ema_indicator(close=data['volume'], window=window)
        # Z-점수_정규화
        data[f'sma_{name}_close_ma{window}_ratio'] = z_score_normalization(data[f'sma_{name}_close_ma{window}'])
        data[f'ema_{name}_close_ma{window}_ratio'] = z_score_normalization(data[f'ema_{name}_close_ma{window}'])
        data[f'sma_{name}_volume_ma{window}_ratio'] = z_score_normalization(data[f'sma_{name}_volume_ma{window}'])
        data[f'ema_{name}_volume_ma{window}_ratio'] = z_score_normalization(data[f'ema_{name}_volume_ma{window}'])

    data[f'{name}_close_diff'] = np.zeros(len(data))
    data.loc[1:, f'{name}_close_diff'] = (data['close'][1:].values - data['close'][:-1].values) / data['close'][:-1].values * 100
    data[f'{name}_volume_diff'] = np.zeros(len(data))
    data.loc[1:, f'{name}_volume_diff'] = (data['volume'][1:].values - data['volume'][:-1].values) / data['volume'][:-1].values * 100

    data = data[['date', f'sma_{name}_close_ma5_ratio', f'sma_{name}_close_ma10_ratio', f'sma_{name}_close_ma20_ratio', f'sma_{name}_close_ma60_ratio', f'sma_{name}_close_ma120_ratio',
                f'ema_{name}_close_ma5_ratio', f'ema_{name}_close_ma10_ratio', f'ema_{name}_close_ma20_ratio', f'ema_{name}_close_ma60_ratio', f'ema_{name}_close_ma120_ratio',
                f'sma_{name}_volume_ma5_ratio', f'sma_{name}_volume_ma10_ratio', f'sma_{name}_volume_ma20_ratio', f'sma_{name}_volume_ma60_ratio', f'sma_{name}_volume_ma120_ratio',
                f'ema_{name}_volume_ma5_ratio', f'ema_{name}_volume_ma10_ratio', f'ema_{name}_volume_ma20_ratio', f'ema_{name}_volume_ma60_ratio', f'ema_{name}_volume_ma120_ratio',
                f'{name}_close_diff', f'{name}_volume_diff']]

    return data

def load_data(code, start_date, end_date):
    # 파이넌스데이터리더 API 데이터 2년치 및 데이터 전처리
    df = ydatareader(code, start_date, end_date)
    # 기간 변환
    df['date'] = df['date'].astype(str)
    df['date'] = df['date'].str.replace('-', '')
    # 차트 데이터 분리
    chart_data = df[COLUMNS_CHART_DATA]
    # 학습 데이터 분리
    training_data = df[COLUMNS_TRAINING_DATA].values

    return chart_data, training_data